In [ ]:
import os
import pandas as pd

In [ ]:
def p(d, max_rows=None, max_cols=None):
    with pd.option_context('display.max_rows', max_rows, 'display.max_columns', max_cols, 'display.float_format', lambda x: f'{x}'):
        display(d)

In [ ]:
df = pd.read_csv(os.path.expanduser('~/dropbox/edw-echo-raw.csv'), low_memory=False)

In [ ]:
# Column Names
# df.keys()

In [ ]:
# Measurement Names
# p(df['ComponentNM'].drop_duplicates())

In [ ]:
peak_velocity_name = "aortic valve peak velocity"
mean_gradient_name = "aortic valve mean gradient"
valve_area_name = "aortic valve area"
indexed_valve_area_name = "indexed aortic valve area"

# maps ComponentNM to a standard name (ComponentCommonNM may differ)
name_map = {
    "AORTIC VALVE PEAK VELOCITY (1)": peak_velocity_name,
    "DOP CALC AO PEAK VEL":           peak_velocity_name,
    "AV PEAK VELOCITY":               peak_velocity_name,
    "AORTIC VALVE MEAN GRADIENT (1)": mean_gradient_name,
    "AV MEAN GRADIENT":               mean_gradient_name,
    "AORTIC VALVE AREA (1)":          valve_area_name,
    "AV VALVE AREA":                  valve_area_name,
    "AORTIC VALVE AREA INDEX (1)":    indexed_valve_area_name,
}

In [ ]:
# only get measurements defined in name_map
as_df = df[df['ComponentNM'].apply(lambda key: key in name_map)].copy()

# label measurements with standard name and make friendlier names
as_df['measurement'] = as_df['ComponentNM'].apply(lambda key: name_map[key])
as_df['value'] = as_df['ResultValueNBR']
as_df['units'] = as_df['ReferenceRangeUnitCD']

# remove exact duplicates measurements (same patient, same echo, same measurement, same value)
as_df = as_df.drop_duplicates(['MRN', 'OrderProcedureID', 'measurement', 'ResultTXT'])

In [ ]:
# for a given echo, there should only be 1 result per measurement (exact duplicates were already removed)
for name, group in as_df.groupby('measurement'):
    dupes = group[group.duplicated('OrderProcedureID', keep=False)]
    if len(dupes) != 0:
        print(f"Duplicate measurements for {name} found, please fix before proceding:")
        print(dupes)
        break

In [ ]:
# reformat data from 1 row per measurement per echo to echo by measurement table
non_pivot_columns = ['MRN', 'PatientID', 'PatientEncounterID', 'ProcedureID', 'ProcedureDSC', 'OrderProcedureID', 'OrderDTS', 'StartDTS', 'EndDTS', 'ResultDTS', 'OrderTypeDSC', 'OrderDisplayNM', 'ComponentObservedDTS', 'SpecimenReceivedTimeDTS', 'SpecimenTakenTimeDTS']
pivot_columns = ['measurement']
pivot_values = ['value', 'units']

data = as_df.pivot(index=non_pivot_columns, columns=pivot_columns, values=pivot_values)
data.columns = data.columns.to_flat_index().map(lambda k: k[1] if k[0] == "value" else f"{k[1]} {k[0]}")
data = data.reset_index()
data

In [ ]:
n_patients = len(data['MRN'].drop_duplicates())
n_echos = len(data[['MRN', 'OrderProcedureID']].drop_duplicates())
n_as = len(data.dropna(subset=[peak_velocity_name, mean_gradient_name, valve_area_name])[['MRN', 'OrderProcedureID']].drop_duplicates())

print(f"Number of Patients:\t\t{n_patients}")
print(f"Number of Echos:\t\t{n_echos}")
print(f"Number of Echos w/ AS values:\t{n_as}")

In [ ]:
data.to_csv(os.path.expanduser('~/dropbox/edw-echo.csv'), index=False)